<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220118_sm_know_submission2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 전공을 임베딩

In [1]:
!pip install gluonnlp
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-by8ebe_j/kobert-tokenizer_f9ce57a75a554401891a653373eb4b4f
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-by8ebe_j/kobert-tokenizer_f9ce57a75a554401891a653373eb4b4f
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-yrxcne6_
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-yrxcne6_


In [2]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
warnings.filterwarnings(action='ignore')

In [73]:
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]
train3 = train3[train3['idx'] != 25823]
train3 = train3[train3['idx'] != 18746]
train3 = train3[train3['idx'] != 19183]
train3 = train3[train3['idx'] != 21879]
train3 = train3[train3['idx'] != 21931]
train3 = train3[train3['idx'] != 26330]

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

test2.loc[3982, test2.columns[1:]] = 0
test3.loc[3694, test3.columns[1:]] = 0
test3.loc[7090, test3.columns[1:]] = 0

In [ ]:
# ## CPU
# # device = torch.device("cpu")

# ## GPU
# device = torch.device("cuda:0")

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [74]:
def text_to_token(text:str):
  temp = tokenizer.encode(text)

  return temp[1]

In [75]:
train3['bq27_1'].replace(' ', '없음', inplace=True)

In [76]:
train3['bq27_1'] = train3['bq27_1'].fillna('없음')

In [77]:
train1['bq38_1'] = train1['bq38_1'].map(text_to_token)
train2['bq37_1'] = train2['bq37_1'].map(text_to_token)
train3['bq27_1'] = train3['bq27_1'].map(text_to_token)
train4['bq26_1'] = train4['bq26_1'].map(text_to_token)

In [78]:
test2.loc[3982, 'bq37_1'] = '없음'
test3.loc[3694, 'bq27_1'] = '없음'
test3.loc[7090, 'bq27_1'] = '없음'

In [79]:
test1['bq38_1'] = test1['bq38_1'].map(text_to_token)
test2['bq37_1'] = test2['bq37_1'].map(text_to_token)
test3['bq27_1'] = test3['bq27_1'].map(text_to_token)
test4['bq26_1'] = test4['bq26_1'].map(text_to_token)

In [80]:
know_train = [train1, train2, train3, train4]
know_test = [test1, test2, test3, test4]

In [81]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [82]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [83]:
years = ['2017', '2018', '2019', '2020']

rf_predicts = []
year_encoder = {}
int_col = {}
text_col = {}

for year, train, test in zip(years, know_train, know_test):
    print(year)
    encoders = {}
    int_temp = []
    text_temp = []
    for col in train.columns:
        if col == 'ID':
            continue
        try:
            train[col] = train[col].map(int)
            int_temp.append(col)
        except:
            encoder = LabelEncoder()
            train[col] = train[col].map(str)
            train[col] = encoder.fit_transform(train[col])
            encoders[col] = encoder
            text_temp.append(col)
    year_encoder[year] = encoders
    int_col[year] = int_temp
    text_col[year] = text_temp

    feature = train.columns.drop(['idx', 'knowcode'])

    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train[feature], train['knowcode'])

    for col in test.columns:
      try:
          test[col] = test[col].map(int)
      except:
          encoder = year_encoder[year][col]
          test[col] = test[col].map(str)
          category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
          test[col] = test[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
    feature = test.columns.drop(['idx'])
    pred = model.predict(test[feature])
    rf_predicts.extend(pred)

2017
2018
2019
2020


In [86]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기

In [87]:
submission['knowcode'] = rf_predicts

In [88]:
submission

,idx,knowcode
0,0,29401
1,1,28402
2,2,612101
3,3,121102
4,4,412003
...,...,...
35226,35244,153104
35227,35245,24402
35228,35246,833001
35229,35247,140101


In [89]:
submission.to_csv('/content/gdrive/MyDrive/know/submission.csv', index=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

n_gram_range = (1, 1)

In [ ]:
# 리뷰하나씩 키워드로 분리(tfidf-vectorizer)
token_review_list=[]

for i in range(len(train1)): # 리뷰하나당 처리하기위해 for문 
    review_vectorized = TfidfVectorizer(ngram_range=n_gram_range).fit([train1['bq19_1'][i]])
    token_review = review_vectorized.get_feature_names()
    
    token_review_list.append(token_review) # 하나의 리스트를 만들어서 df에 추가해야함.
train1['text_tfidf']=token_review_list # 토큰 단위로 나누어진 리뷰저장

ValueError: ignored

In [ ]:
train1['bq19_1']

0                             생산설비의 자동화로
1                             건설 수주가 없어서
2                        신축 건설경기가 좋지 않아서
3                       업무량 감소, 자동화로 인해서
4              부동산 거품 빠지면서 건축 경기도 덩달아 불황
                      ...               
9481                 출판업무 특성상 기획업무는 변함없음
9482    항공우주분야는 국가 경쟁력을 위해 육성해야 되는 분야이므로
9483                          건설경기는 꾸준해서
9484                 공무원의 인원은 크게 변동이 없어서
9485                            업체 과다 경쟁
Name: bq19_1, Length: 9486, dtype: object